## Connect to IB

In [ ]:
from ib_insync import *
util.startLoop()  

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

## Get tick by tick data for HSI trading (RTH & ATH) for 1 month

In [ ]:
import pandas as pd
import time
from datetime import timedelta, datetime

DATE = '20201029'

def getdatesfuture(date):
    # get list of dates 
    todate = datetime.strptime(date, '%Y%m%d')
    delta = timedelta(days=15)    
    list_of_dates = []

    for i in range(delta.days + 1):
        day = todate - delta + timedelta(days=i)
        list_of_dates.append(day.strftime("%Y%m%d"))

    return list_of_dates

contract = Future('HSI', DATE, 'HKFE', includeExpired=True)

df = pd.DataFrame()

list_of_dates = getdatesfuture(DATE)

for i in list_of_dates:
    timing = f'{i} 04:00:00'
    temp = pd.DataFrame({'time': [0]})
    
    while True:
        bars = ib.reqHistoricalTicks(contract, startDateTime= timing, endDateTime='', numberOfTicks=1000, whatToShow='TRADES', useRth=False)
        if bars:
            timing = (bars[-1].time.replace(tzinfo=None) + timedelta(hours=8, seconds=1)).strftime('%Y%m%d %H:%M:%S')
            temp = util.df(bars)[['time','price','size']]

            df = pd.concat([df, temp])

            print(i, timing)
            
        else:
            timing = (datetime.strptime(timing, '%Y%m%d %H:%M:%S') + timedelta(seconds=1)).strftime('%Y%m%d %H:%M:%S')
            
# Have to end manually in the end

## Save as csv

In [ ]:
df.to_csv('10tick.csv', index=False)

## Get 6 months HSI 5sec bar for first 1 hr

In [ ]:
def getdatesfuture(date):
    # get list of dates 
    from datetime import timedelta, datetime
    todate = datetime.strptime(date, '%Y%m%d')
    delta = timedelta(days=35)    
    list_of_dates = []

    for i in range(delta.days + 1):
        day = todate - delta + timedelta(days=i)
        list_of_dates.append(day.strftime("%Y%m%d"))

    return list_of_dates


def getdatafuture(date):
    # get traded price of future
    import pandas as pd

    contract = Future('HSI', date, 'HKFE', includeExpired=True)

    df = pd.DataFrame()

    list_of_dates = getdatesfuture(date)

    for i in list_of_dates:
        bars = ib.reqHistoricalData(contract, endDateTime=f'{i} 09:45:00', durationStr='1800 S', barSizeSetting='1 secs', whatToShow='BID', useRTH=True)
        dfbid = util.df(bars)

        bars = ib.reqHistoricalData(contract, endDateTime=f'{i} 09:45:00', durationStr='1800 S', barSizeSetting='1 secs', whatToShow='ASK', useRTH=True)
        dfask = util.df(bars)

        merged = pd.merge(dfbid,dfask, left_index=True, right_index=True,suffixes=('bid','ask'))
        temp = merged[['datebid', 'openbid', 'highbid', 'lowbid', 'openask', 'highask', 'lowask']]

        bars = ib.reqHistoricalData(contract, endDateTime=f'{i} 09:45:00', durationStr='1800 S', barSizeSetting='1 secs', whatToShow='TRADES', useRTH=True)
        dftrades = util.df(bars)

        merged = pd.merge(temp,dftrades, left_index=True, right_index=True,suffixes=('','trades'))
        temp = merged[['datebid', 'openbid', 'highbid', 'lowbid', 'openask', 'highask', 'lowask', 'open', 'high', 'low', 'close', 'volume', 'average', 'barCount']]

        df = pd.concat([df, temp])
        print(i)
        
        time.sleep(30)  
        
    return(df)

#Get recent 2 years data
import pandas as pd
import time
df = pd.DataFrame()

for year in range(2020,2021):
    for month in range(6,13):
        for day in range(27,31):
            try:
                df = df.append(getdatafuture(f'{year}{month+1:02d}{day}'))
            except:
                pass

## Get HSI 30secs bar inside and outside RTH

In [ ]:
def getdatesfuture(date):
    # get list of dates 
    from datetime import timedelta, datetime
    todate = datetime.strptime(date, '%Y%m%d')
    delta = timedelta(days=35)    
    list_of_dates = []

    for i in range(delta.days + 1):
        day = todate - delta + timedelta(days=i)
        list_of_dates.append(day.strftime("%Y%m%d"))

    return list_of_dates


def getdatafuture(date):
    # get traded price of future
    import pandas as pd

    contract = Future('HSI', date, 'HKFE', includeExpired=True)

    df = pd.DataFrame()

    list_of_dates = getdatesfuture(date)

    for i in list_of_dates:

        bars = ib.reqHistoricalData(contract, endDateTime=f'{i} 16:30:00', durationStr='1 D', barSizeSetting='30 secs', whatToShow='TRADES', useRTH=False)
        temp = util.df(bars)
        temp = temp[['date', 'open', 'high', 'low', 'close', 'volume', 'average', 'barCount']]
        
        try:
            check = df.iloc[-1].equals(temp.iloc[-1])
        except:
            check = False

        if not check:
            df = pd.concat([df, temp])
            print(i)
                
    return(df)

#Get recent 2 years data
import pandas as pd
import time
df = pd.DataFrame()

for year in range(2019,2021):
    for month in range(1,13):
        for day in range(25,31):
            try:
                df = df.append(getdatafuture(f'{year}{month+1:02d}{day}'))
            except:
                pass